# Обучаем Qwen

## Установка зависимостей

In [ ]:
!git clone --depth 1 https://github.com/hiyouga/LLaMA-Factory.git

In [ ]:
!pip uninstall -y jax
!pip install -e .[bitsandbytes,liger-kernel]
!pip install pandas
!pip install huggingface_hub
# Если в docker нету torch
# !pip install torch==2.3.1 torchvision==0.18.1 torchaudio==2.3.1
# !pip install -e ".[torch,metrics]"

In [ ]:
# Лучше не трогать
%cd LLaMA-Factory

In [ ]:
!pip install -r requirements.txt
!pip install llamafactory

In [ ]:
# Лучше не трогать
%cd ..

Проверяем что нам доступен torch

In [ ]:
import torch
try:
  assert torch.cuda.is_available() is True
except AssertionError:
  print("Please set up a GPU before using LLaMA Factory: https://medium.com/mlearning-ai/training-yolov4-on-google-colab-316f8fff99c6")

## Поменять при необходимости

Указываем модель и для какой задачи будет использоваться

**task_description должен совпадать с названием датасета**

In [ ]:
model="Qwen/Qwen2.5-0.5B-Instruct"
task_description = "matching"

## Устанавливаем модель

Обязательно делаем это тут, а не в LLaMA Factory, так как там это синхроно

In [ ]:
from huggingface_hub import login, snapshot_download

login("hf_token")

snapshot_download(model)

Папка для сохранения результата

In [ ]:
from datetime import datetime
import os

timestamp = datetime.now().strftime("%Y-%m-%d-%H:%M:%S")
output_dir = f'/home/jupyter/lora/{model}/{task_description}/train_{timestamp}'
# os.makedirs(output_dir, exist_ok=True)
print(output_dir)

## Запускаем обучение

Меняем на нужные параметры запускаем

In [ ]:
step = 1
batch_size = 1
max_prompt_size = 100
lora_rank = 8
lora_alpha = 16
lora_dropout = 0
save_steps = 5000


command = f"""
llamafactory-cli train \
    --stage sft \
    --do_train True \
    --model_name_or_path {model} \
    --preprocessing_num_workers 16 \
    --finetuning_type lora \
    --template qwen \
    --flash_attn auto \
    --dataset_dir ./ \
    --dataset {task_description} \
    --cutoff_len {max_prompt_size} \
    --learning_rate 5e-05 \
    --num_train_epochs 3.0 \
    --max_samples {step} \
    --per_device_train_batch_size {batch_size} \
    --gradient_accumulation_steps 8 \
    --lr_scheduler_type cosine \
    --max_grad_norm 1.0 \
    --logging_steps 5 \
    --save_steps {save_steps} \
    --warmup_steps 0 \
    --packing False \
    --report_to none \
    --output_dir {output_dir} \
    --bf16 True \
    --plot_loss True \
    --ddp_timeout 180000000 \
    --optim adamw_torch \
    --lora_rank {lora_rank} \
    --lora_alpha {lora_alpha} \
    --lora_dropout {lora_dropout} \
    --lora_target all
"""

# Выполняем команду
get_ipython().system(command)


In [ ]:
!pwd

In [ ]:
!GRADIO_SHARE=1 llamafactory-cli webui

In [ ]:
import os
import shutil

for dirpath, dirnames, filenames in os.walk(output_dir, topdown=False):
        for dirname in dirnames:
            dir_to_remove = os.path.join(dirpath, dirname)
            shutil.rmtree(dir_to_remove)
            print(f"Удалена папка и её содержимое: {dir_to_remove}")

In [ ]:
print('/'.join(output_dir.split('/')[-4:-1]))

In [ ]:
command = f"""
mc mv {output_dir} minio/workspace/models/lora/{'/'.join(output_dir.split('/')[-4:-1])} --recursive
"""
print(command)
get_ipython().system(command)
